# Исследование надежности заемщиков.

Закачиком (кредитный отдел банка) поставлена задача исследовать зависимость платежеспособности клиента от следующих факторов: 
* количество детей клиента;
* семейное положение клиента;
* уровень дохода клиента;
* цель кредита.

Входные данные от банка — статистика о платёжеспособности клиентов.
При выполнении исследования крайне необходимо выполнить предобработку полученных данных, а именно:
* выполнить проверку на пропуски;
* исследовать данные на аномалии;
* проверить дубликаты.

### Шаг 1. Обзор данных

In [1]:
# импорт библиотеки pandas:
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df:
df = pd.read_csv('/datasets/data.csv')

In [3]:
# получение первых 10 строк таблицы df:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице df:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице одиннадцать столбцов, ниже их описание:
    
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;  
* `purpose` — цель получения кредита;

В названиях колонок нет нарушений стиля.
Однако, количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

### Шаг 2.1 Заполнение пропусков

In [5]:
# подсчёт пропусков:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Итак, в двух колонках `days_employed` и `total_income` по 2174 пропущенных значений.
Колонка `days_employed` имеет аномалии (отрицательное количество рабочих дней), поэтому потребует дополнительной обработки.

Колонка `total_income` отображает доход потенциальных заемщиков. Рассчитаем долю пропущенных значений:

In [6]:
# разделим количество пропущенных значений в колонке на количество строк в таблице:
df['total_income'].isna().sum() / df.shape[0]

0.10099883855981417

Доля пропущенных значений порядка 10%, что может существенно повлиять на результаты исследований. Пропуски в данных появляются разными путями. В данном случае, скорее всего, пользователи не указали сведения о доходе или автоматизированная система сбора информации дала сбой.

Доход — количественное значение. Пропуски в таких колонках обычно заполняют характерными значениями: средним арифметическим или медианой. В случае с заработком лучше использовать медиану, ведь в данных могут быть выдающиеся значения, которые сместят среднее. Заполнять пропуски нулём или максимальным значением не стоит — такие значения нельзя назвать характерными. Заполним пропуски в стобце `total_income` медианным значением:

In [7]:
# рассчитаем медианный доход:
total_income_medium = df['total_income'].median()
total_income_medium

145017.93753253992

Неплохие доходы у заявителей, раз в десять больше прожиточного минимума в РФ.

Проверим, какой минимальный и какой максимальный доход у клиентов:

In [8]:
# рассчитаем минимальный доход:
total_income_min = df['total_income'].min()
print('Минимальный доход среди клиентов', int(total_income_min))

Минимальный доход среди клиентов 20667


In [9]:
# рассчитаем максимальный доход:
total_income_max = df['total_income'].max()
print('Максимальный доход среди клиентов', int(total_income_max))

Максимальный доход среди клиентов 2265604


In [10]:
#заменим пропущенные значения в столбце 'total income' на полученное медианное значение:
df['total_income'] = df['total_income'].fillna(total_income_medium)

Выполним проверку стобца `total_income` на пропуски после замены:

In [11]:
df['total_income'].isna().sum()

0

Пропущенных значений в столбце `total_income` больше нет.

### Шаг 2.2 Проверка данных на аномалии и исправления.

В столбцах с числовыми типами данных (int и float) могут встречаться различные аномалии и выбросы, которые необходимо исправить для качественного анализа данных. Требуется проверить следующие столбцы:
* `children`
* `days_employed`
* `dob_years`
* `education_id`
* `family_status_id`
* `debt`
* `total_income`


#### Проверка данных столбца 'children'

Выполним проверку столбца `children` на аномалии:

In [12]:
df.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Имеем отрицательное значение детей, что скорее всего является ошибкой.

In [13]:
# применим функцию abs для возврата модуля чисел в указанной колонке:
df['children'] = df['children'].abs()

In [14]:
df.groupby('children')['children'].count()

children
0     14149
1      4865
2      2055
3       330
4        41
5         9
20       76
Name: children, dtype: int64

Также у 76 заявителей по 20 детей, что в теории возможно, но скорее всего и тут некая ошибка, т.к. имеет место тенденция к снижению количества клиентов при увеличении количества детей.

In [15]:
# расчет доли заявителей, у которых якобы 20 детей:
children_20 = len(df[df['children']==20]) / len(df['children'])
print(f'Доля таких заявителей {children_20:.2%} от общего количества заявителей') 

Доля таких заявителей 0.35% от общего количества заявителей


Выведем таблицу с заявителями, у которых 20 детей.

In [16]:
df[df['children']==20].sort_values(by='dob_years').head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
5315,20,-2047.754733,24,среднее,1,женат / замужем,0,F,сотрудник,0,100415.236833,покупка коммерческой недвижимости
12909,20,NaN,25,среднее,1,женат / замужем,0,M,сотрудник,0,145017.937533,операции с жильем
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью
14474,20,-2625.072839,27,среднее,1,женат / замужем,0,F,сотрудник,0,244092.075999,высшее образование
21491,20,-173.954460,27,среднее,1,женат / замужем,0,F,сотрудник,0,155520.899688,на покупку подержанного автомобиля
16795,20,-2530.540521,29,среднее,1,женат / замужем,0,M,компаньон,0,95454.865761,недвижимость
16371,20,-419.894814,29,неоконченное высшее,2,Не женат / не замужем,4,M,сотрудник,0,122062.334283,операции с коммерческой недвижимостью


Осмелюсь предположить, что здесь вместо 20 детей имелось в виду по 2 детей, а лишний ноль - это какой-то сбой системы сбора данных. Маловероятно, чтобы девушки от 23 до 27 лет могли иметь по 20 детей. Выполним необходимую замену.

In [17]:
# замена стандартным методом replace():
df['children'] = df['children'].replace(20, 2)

In [18]:
df.groupby('children')['children'].count()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

#### Проверка данных столбца 'days_employed'

Ранее было указано, что в столбце `days_employed` наблюдаются отрицательные значения. Выведем датафрейм с отрицательными значениями и отсортируем по возрастанию

In [19]:
df[df['days_employed'] < 0].sort_values(by='days_employed')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
2127,1,-34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования
9683,0,-33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
6157,2,-30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
8336,0,-24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования


Предположительно, случился программный сбой, который начал присваивать дням отрицательные значения, возможен также человеческий фактор. В целом, значения трудового стажа в днях выглядят адекватными для возраста заявителей.

In [20]:
# применим функцию abs для возврата модуля чисел в указанной колонке:
df['days_employed'] = df['days_employed'].abs()

Выполним проверку на минимальные и максимальные значения:

In [21]:
print('Минимальный общий трудовой стаж в днях:', int(df['days_employed'].min()))

Минимальный общий трудовой стаж в днях: 24


In [22]:
print('Максимальный общий трудовой стаж в днях:', int(df['days_employed'].max()))

Максимальный общий трудовой стаж в днях: 401755


In [23]:
print('Максимальный общий трудовой стаж в годах:', int(df['days_employed'].max() / 365))

Максимальный общий трудовой стаж в годах: 1100


Минимальное количество общего трудового стажа в годах составляет 24 дня, что вполне приемлемо. Однако, максимальный общий трудовой стаж в годах составил 1100 лет, что невозможно. Предположим, что на пенсию выходят в 65-70, а начинают работать в 20, значит работают порядка 45-50 лет, т.е. 16425-18250 дней соответственно. Выделим строки датафрейма по количеству дней > 17000 и отсортируем по возрастанию.

In [24]:
df[df['days_employed'] > 17000].sort_values(by='days_employed')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4299,0,17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
16335,1,18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования


Начиная с 3 строки выделенного датафрейма (строка #20444) наблюдается значение общего трудового стажа 328728 дней (900 лет). Предположительно, случился программный сбой, и стаж был переведен в часы. Необходимо это исправить:

In [25]:
# функция для замены трудового стажа на дни по условию:
def days_employed_replace(row):
    days_employed = row['days_employed']
    if days_employed > 300000:
        return (days_employed / 24)
    else:
        return days_employed
    
# применение функции для замены трудового стажа на дни:
df['days_employed'] = df.apply(days_employed_replace, axis=1)    

Выполним проверку работы функции:

In [26]:
df[df['days_employed'] > 17000].sort_values(by='days_employed')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4299,0,17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
16335,1,18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью


Наблюдаем, что больше нет аномально высоких значений трудового стажа. Также проверим строку #20444 вызвав атрибут loc: 

In [27]:
# изначальное значение в строке #20444 равно 328728.720605, вручную разделим на 24:
k_expected = 328728.720605 / 24
# фактическое значение в строке #20444 после обработки функцией days_employed_replace():
k_fact = df.loc[20444, 'days_employed']
if round(k_expected) == round(k_fact):
    print (f'Проверка успешная, {round(k_expected)} равно {round(k_fact)}')
else:
    print (f'Ошибка, {round(k_expected)} не равно {round(k_fact)}')


Проверка успешная, 13697 равно 13697


Проверим датафрейм на пропущенные значения еще раз:

In [28]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Количество рабочих дней количественное значение. Пропуски в таких колонках обычно заполняют характерными значениями: средним арифметическим или медианой. В данном случае также лучше использовать медиану, ведь в данных могут быть выдающиеся значения, которые сместят среднее (разброс количества рабочих дней очень большой). Заполнять пропуски нулём или максимальным значением не стоит — такие значения нельзя назвать характерными. Заполним пропуски в стобце `days_employed` медианным значением:

In [29]:
# рассчитаем медианное количество рабочих дней:
days_employed_medium = df['days_employed'].median()
days_employed_medium

2194.220566878695

In [30]:
# или количество трудового стажа в годах:
days_employed_medium / 365

6.0115631969279315

Трудовой стаж типичного потенциального заемщика - 6 лет.

In [31]:
#заменим пропущенные значения в столбце 'days_employed' на полученное медианное значение:
df['days_employed'] = df['days_employed'].fillna(days_employed_medium)

Выполним проверку стобца `days_employed` на пропуски после замены:

In [32]:
df['days_employed'].isna().sum()

0

Пропущенных значений в столбце `days_employed` больше нет.

#### Проверка данных столбца 'dob_years'

Выполним проверку столбца `dob_years` на аномалии:

In [33]:
df.groupby('dob_years')['dob_years'].count()

dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

У 101 клиента указан возраст 0 лет, скорее всего ошибка при заполнении данных. Пропусков не обнаружено. Предлагается заменить эти значения на средний возраст, т.е. применить метод mean().

In [34]:
# рассчитаем средний возраст:
dob_years_mean = df['dob_years'].mean()
# переведем в целое число:
dob_years_mean = int(dob_years_mean)
print(f'Cредний возраст заявителя {dob_years_mean} года')

Cредний возраст заявителя 43 года


In [35]:
# замена стандартным методом replace()
df['dob_years'] = df['dob_years'].replace(0, dob_years_mean)

In [36]:
# проверка замены 
df.groupby('dob_years')['dob_years'].count()

dob_years
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    614
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Клиентов с возрастом 0 лет более нет в датафрейме, зато клиентов в возрасте 43 лет стало больше на 101 человека.

#### Проверка данных столбца 'education_id'

Рассмотрим столбец `education_id`:

In [37]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Каких-либо сомнительных значений нет, это просто идентификаторы уровня образования.

#### Проверка данных столбца 'family_status_id'

Рассмотрим столбец `family_status_id`:

In [38]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Каких-либо сомнительных значений нет, это просто идентификаторы семейного положения.

#### Проверка данных столбца 'debt'

Рассмотрим столбец `debt`:

In [39]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Всего в столбце два значения - 0 и 1, что означает у клиента 'нет задолженности' и 'есть задолженность' соответственно.

#### Проверка данных столбца 'total_income'

Отсортируем столбец `total_income` по возрастанию дохода клиентов (ранее уже определяли минимальный и максимальный доход):

In [40]:
df['total_income'].sort_values()

14585    2.066726e+04
13006    2.120528e+04
16174    2.136765e+04
1598     2.169510e+04
14276    2.189561e+04
             ...     
17178    1.711309e+06
20809    1.715018e+06
9169     1.726276e+06
19606    2.200852e+06
12412    2.265604e+06
Name: total_income, Length: 21525, dtype: float64

Никаких явных отклонений от допустимых значений дохода не наблюдается. Однако считывать такие данные крайне неудобно, предлагается перевести тип данных в указанном столбце в целочисленный.

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце `total_income` на целочисленный:

In [41]:
# применим метод astype к столбцу 'total_income'
df['total_income'] = df['total_income'].astype('float32')
df['total_income'].sort_values()

14585    2.066726e+04
13006    2.120528e+04
16174    2.136765e+04
1598     2.169510e+04
14276    2.189562e+04
             ...     
17178    1.711309e+06
20809    1.715018e+06
9169     1.726276e+06
19606    2.200852e+06
12412    2.265604e+06
Name: total_income, Length: 21525, dtype: float32

### Шаг 2.4. Удаление дубликатов.

Выполним обработку датафрейма на предмет удаления как явных, так и неявных дубликатов.

В первую очередь, необходимо проверить столбцы с типом данных `object` на неявные дубликаты с учетом регистра. Проверка потребуется для следующих столбцов: 
* `education`
* `family_status`
* `gender`
* `income_type`
* `purpose`

#### Проверка дубликатов в столбце 'education'

Рассмотрим столбец `education`:

In [42]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Есть несколько одинаковых строк, отличающихся регистром. Чтобы учесть такие дубликаты, все символы в строке привем к нижнему регистру вызовом метода str.lower() и перезапишем колонку `education`:  

In [43]:
df['education'] = df['education'].str.lower() 
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Больше всего людей со средним образованием, раза в три меньше людей с высшим.

#### Проверка дубликатов в столбце 'family_status'

Рассмотрим столбец `family_status`:

In [44]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В целом дубликатов не наблюдается, но для единообразия приведем все к нижнему регистру:

In [45]:
df['family_status'] = df['family_status'].str.lower() 
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

#### Проверка дубликатов в столбце 'gender'

Выполним проверку столбца `gender`:

In [46]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Дубликатов также нет. Есть непонятный пол XNA в количестве одной штуки, на результаты исследования повлиять не должно. Ради интереса вызовем строку с таким полом.

In [47]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.15625,покупка недвижимости


#### Проверка дубликатов в столбце 'income_type'

Выполним проверку столбца `income_type`:

In [48]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Дубликатов нет, регистр нижний. Дальнейших действий не требуется.

#### Проверка дубликатов в столбце 'purpose'

Выполним проверку столбца `purpose`:

In [49]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Довольно много разнообразных целей получения кредита, на мой взгляд возникает необходимость привести категории к единообразию.

#### Удаление явных дубликатов

После полной обработки данных необходимо проверить данные на явные дубликаты и удалить их. 

In [50]:
# подсчёт явных дубликатов
df.duplicated().sum()

71

71 явный дубликат, который можно удалить путем применения специального метода `pandas`:

In [51]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True) 

In [52]:
# проверка на отсутствие явных дубликатов
df.duplicated().sum()

0

Для удаления дубликатов использовалось несколько методов. Первый - использование функции drop_duplicates() для удаления явных дубликатов. Второй - вызов функции str.lower() для приведения категорий к нижнему регистру, т.к. заполняют как строчными, так и прописными буквами. Причиной появления таких дубликатов является скорее всего человеческий фактор, а также возможная недоработка со стороны разработчиков (можно сделать выпадающий список строгих категорий, а также принудительно переводить все сразу в нижний регистр).

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Формирование датафрейма `education_dict`, который будет являться словарем для столбца `education`:

In [53]:
education_dict = df[['education_id', 'education']]

Выведем таблицу, в которой `education_id` соответствует `education`

In [54]:
education_dict_desc = education_dict.drop_duplicates().reset_index(drop=True)
education_dict_desc

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Формирование датафрейма `family_status_dict`, который будет являться словарем для столбца `family_status`:

In [55]:
family_status_dict = df[['family_status_id', 'family_status']]

Выведем таблицу, в которой `family_status_id` соответствует `family_status`

In [56]:
family_status_dict_desc = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict_desc

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`. В обновленном датафрейме будем обращаться не к удаленным текстовым строкам, а к их числовым обозначениям, что позволит ускорить обработку данных и упростит визуальную работу с таблицей.

In [57]:
df = df.drop(['education', 'family_status'], axis='columns')
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875.640625,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080.015625,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885.953125,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628.562500,дополнительное образование
4,0,14177.753002,53,1,1,F,пенсионер,0,158616.078125,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763.562500,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525.968750,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823.937500,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856.835938,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425.937500,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

Формирование столбца `total_income_category` с категориями по доходу получателей кредита:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [58]:
# функция для определения категории по доходу:
def total_income_category_group(total_income):
    if total_income <= 30000:
        return('E')
    elif total_income >= 30001 and total_income <=50000:
        return('D')
    elif total_income >= 50001 and total_income <=200000:
        return('C')
    elif total_income >= 200001 and total_income <=1000000:
        return('B')
    elif total_income >= 1000001:
        return('A')

In [59]:
# добавление нового столбца с категориями доходов 'total_income_category':
df['total_income_category'] = df['total_income'].apply(total_income_category_group)

Выведем первые 10 строк обновленного датафрейма

In [60]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875.640625,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080.015625,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885.953125,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628.562500,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616.078125,сыграть свадьбу,C
5,0,926.185831,27,0,1,M,компаньон,0,255763.562500,покупка жилья,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525.968750,операции с жильем,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823.937500,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856.835938,на проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425.937500,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Как было указано раннее, в столбце `purpose` множество разнообразных целей кредита, которые нуждаются в строгой категоризации.
Необходимо сформировать новый столбец `purpose_category` со следующими категориями:
* операции с автомобилем
* операции с недвижимостью
* проведение свадьбы
* получение образования

Чтобы создать новый столбец, напишем функцию `purpose_category_group()` с параметром `row`.
Функция должна заменить цели кредита из столбца `purpose` на указанные выше строгие категории в новом столбце `purpose_category`.

In [61]:
# просмотр целей кредита для внесения в функцию purpose_category_group(row):
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [62]:
# функция для замены целей получения кредита на строгие категории:
def purpose_category_group(row):
    purpose = row['purpose']
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образ' in purpose:
        return 'получение образования'
    else:
        return 'операции с недвижимостью'

In [63]:
# применение функции для создания нового столбца со строгими категориями целей кредита:
df['purpose_category'] = df.apply(purpose_category_group, axis=1)

In [64]:
# проверка столбца 'purpose_category'
df['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Подготовим сводную таблицу, в которой сгруппируем данные по количеству детей (index='children'), по наличию задолженностей (columns='debt') и подсчитаем количество строк в произвольном столбце путем применения функции count():

In [65]:
children_pivot = df.pivot_table(index='children', columns='debt', values='dob_years', aggfunc='count')

In [66]:
children_pivot

debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Анализ необходимо выполнить путем сравнения относительных величин, а именно отношения количества должников к общему количеству клиентов. Необходимо дополнить сводную таблицу:

In [67]:
children_pivot['debtors, %'] = round((children_pivot[1] / (children_pivot[0] + children_pivot[1])) * 100, 1)

In [68]:
children_pivot

debt,0,1,"debtors, %"
children,,,
0,13028.0,1063.0,7.5
1,4410.0,445.0,9.2
2,1926.0,202.0,9.5
3,303.0,27.0,8.2
4,37.0,4.0,9.8
5,9.0,NaN,NaN


##### Вывод 1:

Анализ относительных величин в сводной таблице `children_pivot` показывает следующее: 
1. нет каких-либо задолженностей у клиентов с 5 детьми, однако таких клиентов всего 9, что не позволяет нам принимать данный факт за аксиому (появление хотя бы одного должника увеличивает долю должников сразу до 10%).
2. среди бездетных клиентов 7,5% должников, при увеличении количества до 1 и 2 количество должников растет до 9,2 и 9,5% соответственно; в целом это похоже на правду, т.к. при появлении детей значительно увеличиваются расходы.
3. однако среди клиентов с 3 детьми количество должников 8,2% (меньше на 1%, чем у клиентов с одним ребенком), возможно это действие государственной поддержки многодетных семей или нерепрезентативная выборка.
4. среди клиентов с 4 детьми должников 9,8%, что вполне укладывается в общий тренд увеличения количества должников при увеличении количества детей.

Заключение: в целом наблюдается тренд увеличения количества должников при увеличении количества детей.

In [69]:
children_pivot_1 = df.pivot_table(index='children', values='debt', aggfunc=['sum','count','mean'])

In [70]:
children_pivot_1

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,0.075438
1,445,4855,0.091658
2,202,2128,0.094925
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Вызовем таблицу `family_status_dict_desc` с описанием ключей семейного положения:

In [71]:
family_status_dict_desc

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Подготовим сводную таблицу, в которой сгруппируем данные по семейному положению (index='family_status_id'), по наличию задолженностей (columns='debt') и подсчитаем количество строк в произвольном столбце путем применения функции count():

In [72]:
family_status_pivot = df.pivot_table(index='family_status_id', columns='debt', values='dob_years', aggfunc='count')


In [73]:
family_status_pivot

debt,0,1
family_status_id,,
0,11408,931
1,3763,388
2,896,63
3,1110,85
4,2536,274


Анализ необходимо выполнить путем сравнения относительных величин, а именно отношения количества должников к общему количеству клиентов. Необходимо дополнить сводную таблицу:

In [74]:
family_status_pivot['debtors, %'] = round((family_status_pivot[1] / (family_status_pivot[0] + family_status_pivot[1])) * 100, 1)

Применим метод merge() к таблице `family_status_pivot` объединив ее с таблицей `family_status_dict_desc`:

In [75]:
family_status_pivot = family_status_pivot.merge(family_status_dict_desc, on='family_status_id', how='left')

In [76]:
family_status_pivot

,family_status_id,0,1,"debtors, %",family_status
0,0,11408,931,7.5,женат / замужем
1,1,3763,388,9.3,гражданский брак
2,2,896,63,6.6,вдовец / вдова
3,3,1110,85,7.1,в разводе
4,4,2536,274,9.8,не женат / не замужем


##### Вывод 2:

Анализ относительных величин в сводной таблице `family_status_pivot` показывает следующее: 
1. меньше всего должников среди вдовцов/вдов, возможно играет роль полученное наследство.
2. среди разведенных клиентов 7,1% должников, среди клиентов в официальном браке 7,5% должников, что сопоставимо с разведенными.
3. среди клиентов, находящихся в т.н. "гражданском браке", должников 9,3%.
4. среди неженатых/незамужних клиентов больше всего должников, а именно 9,8%.
5. основная часть клиентов находятся в семейном положении "женат/замужем".

Заключение: среди клиентов находящихся или находившихся в официальных отношениях меньше всего должников, а неженатые/незамужние клиенты и клиенты в гражданском браке более склоны к задолженностям по возврату кредитов.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Подготовим сводную таблицу, в которой сгруппируем данные по количеству категории дохода (index='total_income_category'), по наличию задолженностей (columns='debt') и подсчитаем количество строк в произвольном столбце путем применения функции count()

In [77]:
total_income_pivot = df.pivot_table(index='total_income_category', columns='debt', values='dob_years', aggfunc='count')

In [78]:
total_income_pivot

debt,0,1
total_income_category,,
A,23,2
B,4685,356
C,14656,1360
D,329,21
E,20,2


Анализ необходимо выполнить путем сравнения относительных величин, а именно отношения количества должников к общему количеству клиентов. Необходимо дополнить сводную таблицу:

In [79]:
total_income_pivot['debtors, %'] = round((total_income_pivot[1] / (total_income_pivot[0] + total_income_pivot[1])) * 100, 1)

In [80]:
total_income_pivot

debt,0,1,"debtors, %"
total_income_category,,,
A,23,2,8.0
B,4685,356,7.1
C,14656,1360,8.5
D,329,21,6.0
E,20,2,9.1


##### Вывод 3:

Анализ относительных величин в сводной таблице `total_income_pivot` показывает следующее: 
1. меньше всего должников в категории D - 6% (доход 30001–50000) и категории B - 7,1% (доход 200001–1000000).
2. среднее количество должников (8%) среди клиентов с самым высоким доходом в категории A (доход 1000001 и выше).
3. в самой многочисленной категории C (доход 50001–200000) должников 8,5%.
4. больше всего должников в самой малочисленной группе E (доход 0–30000) - 9,1%.
5. категории A и C имеют малое количество клиентов, что не позволяет с уверенностью говорить о репрезентативности данных.

Заключение: клиенты с самыми низкими доходами (категория E) больше всего склонны к задолженностями по кредитам, однако при повышении уровня доходов до следующей категории D наблюдается резкий спад среди должников; тем не менее, среди клиентов с большим уровнем доходов количество должников увеличивается. 

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Подготовим сводную таблицу, в которой сгруппируем данные по количеству категории дохода (index='purpose_category'), по наличию задолженностей (columns='debt') и подсчитаем количество строк в произвольном столбце путем применения функции count()

In [81]:
purpose_category_pivot = df.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc='count')

In [82]:
purpose_category_pivot

debt,0,1
purpose_category,,
операции с автомобилем,3903,403
операции с недвижимостью,10029,782
получение образования,3643,370
проведение свадьбы,2138,186


Анализ необходимо выполнить путем сравнения относительных величин, а именно отношения количества должников к общему количеству клиентов. Необходимо дополнить сводную таблицу:

In [83]:
purpose_category_pivot['debtors, %'] = round((purpose_category_pivot[1] / (purpose_category_pivot[0] + purpose_category_pivot[1])) * 100, 1)

In [84]:
purpose_category_pivot

debt,0,1,"debtors, %"
purpose_category,,,
операции с автомобилем,3903,403,9.4
операции с недвижимостью,10029,782,7.2
получение образования,3643,370,9.2
проведение свадьбы,2138,186,8.0


##### Вывод 4:

Анализ относительных величин в сводной таблице `purpose_category_pivot` показывает следующее: 
1. меньше всего должников в категории "операции с недвижимостью", а именно 7,2% (жилье - это базовая потребность, за которое приходится платить).
2. должников в категории "проведение свадьбы" 8%, что несколько больше, чем в категории "операции с недвижимостью".
3. в категориях "получение образования" и "операции с автомобилем" должников примерно равное количество - 9,2% и 9,4% соответственно.
4. "операции с недвижимостью" самая многочисленная категория.
5. категории "получение образования" и "операции с автомобилем" сопоставимы по численности.


Заключение: клиенты, оформляющие кредиты на операции с недвижимостью (ремонт, покупка жилья и т.п.), самые добросовестные по кредитным платежам; клиенты в категориях "получение образования" и "операции с автомобилем" допускают просрочки по кредитам более всего.

## Общий вывод:

Полученная статистика о платёжеспособности клиентов включает в себя следующие данные:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;  
* `purpose` — цель получения кредита;

Для проведения качественного анализа данных выполнена их предобработка, а именно:
* заполнение пропущенных значений в столбцах `days_employed` и `total_income` медианными значениями;
* проверка данных на аномалии и выбросы, в столбцы `children`, `days_employed`, `dob_years` внесены необходимые исправления;
* проверка на явные и неявные дубликаты с учетом нижнего регистра, в столбец `education` внесены необходимые исправления.

Сформирован столбец `total_income_category` с категориями по доходу получателей кредита:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

С целью категоризации целей кредита сфомирован столбец `purpose_category` со следующими категориями:
* операции с автомобилем;
* операции с недвижимостью;
* проведение свадьбы;
* получение образования.

Выполнен комплексный анализ данных представленных данных, ниже представлены результаты проведенного анализа для формирования модели кредитного скоринга: 
1. в целом наблюдается тренд увеличения количества должников при увеличении количества детей.
2. среди клиентов находящихся или находившихся в официальных отношениях меньше всего должников, а неженатые/незамужние клиенты и клиенты в гражданском браке более склоны к задолженностям по возврату кредитов.
3. клиенты с самыми низкими доходами (категория E) больше всего склонны к задолженностями по кредитам, однако при повышении уровня доходов до следующей категории D наблюдается резкий спад среди должников; тем не менее, среди клиентов с большим уровнем доходов количество должников увеличивается. 
4. клиенты, оформляющие кредиты на операции с недвижимостью (ремонт, покупка жилья и т.п.), самые добросовестные по кредитным платежам; клиенты в категориях "получение образования" и "операции с автомобилем" допускают просрочки по кредитам более всего.